## Sprint7 リクルート　飲食店の来客者数予測


店舗データ、過去の来客者数、予約者数データからその後の店舗の来客者数を予測する。
（５月はGWがあるから注意）

出力
８２１店舗(air_id)３９日分　（２０１７年４月２３日〜２０１７年５月３１日）

フロー
- １日目　EDA、かんたんなモデルで出力できるように
- ２、3日目　項目追加、他のカーネルを参考にしながら精度を上げていく

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [76]:
air_reserve = pd.read_csv("../input/air_reserve.csv.zip")
air_store_info = pd.read_csv("../input/air_store_info.csv.zip")
air_visit_data = pd.read_csv("../input/air_visit_data.csv.zip")
date_info = pd.read_csv("../input/date_info.csv.zip")
hpg_reserve = pd.read_csv("../input/hpg_reserve.csv.zip")
hpg_store_info = pd.read_csv("../input/hpg_store_info.csv.zip")
sample_submission = pd.read_csv("../input/sample_submission.csv.zip")
store_id_relation = pd.read_csv("../input/store_id_relation.csv.zip")

### 提出ファイルの形式確認

In [3]:
sample_submission.head()

,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,0
1,air_00a91d42b08b08d9_2017-04-24,0
2,air_00a91d42b08b08d9_2017-04-25,0
3,air_00a91d42b08b08d9_2017-04-26,0
4,air_00a91d42b08b08d9_2017-04-27,0


id列の１９文字まででユニークな要素をカウントすると店舗数がわかる

In [4]:
sample_submission['id'].map(lambda x : x[:20]).nunique()

821

id列の２１文字以降でユニークな要素をカウントすると日数がわかる

In [7]:
sample_submission['id'].map(lambda x : x[21:]).nunique()

39

よって、idは821店舗分、それぞれ39日分のデータを予測しなければならない。

## EDA

In [51]:
air_reserve

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5
5,air_db80363d35f10926,2016-01-02 01:00:00,2016-01-01 16:00:00,2
6,air_db80363d35f10926,2016-01-02 01:00:00,2016-01-01 15:00:00,4
7,air_3bb99a1fe0583897,2016-01-02 16:00:00,2016-01-02 14:00:00,2
8,air_3bb99a1fe0583897,2016-01-02 16:00:00,2016-01-01 20:00:00,2
9,air_2b8b29ddfd35018e,2016-01-02 17:00:00,2016-01-02 17:00:00,2


欠損値なし

In [59]:
air_reserve.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92378 entries, 0 to 92377
Data columns (total 4 columns):
air_store_id        92378 non-null object
visit_datetime      92378 non-null object
reserve_datetime    92378 non-null object
reserve_visitors    92378 non-null int64
dtypes: int64(1), object(3)
memory usage: 2.8+ MB


最多は100人だが平均・分散から見るとほとんどが４、５人以下

In [60]:
air_reserve.describe()

,reserve_visitors
count,92378.000000
mean,4.481749
std,4.919669
min,1.000000
25%,2.000000
50%,3.000000
75%,5.000000
max,100.000000


In [52]:
air_store_info

,air_store_id,air_genre_name,air_area_name,latitude,longitude
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599
5,air_99c3eae84130c1cb,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599
6,air_f183a514cb8ff4fa,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599
7,air_6b9fa44a9cf504a1,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599
8,air_0919d54f0c9a24b8,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599
9,air_2c6c79d597e48096,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599


欠損値なし

In [61]:
air_store_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 829 entries, 0 to 828
Data columns (total 5 columns):
air_store_id      829 non-null object
air_genre_name    829 non-null object
air_area_name     829 non-null object
latitude          829 non-null float64
longitude         829 non-null float64
dtypes: float64(2), object(3)
memory usage: 32.5+ KB


緯度経度共に範囲外の値はない

In [62]:
air_store_info.describe()

,latitude,longitude
count,829.000000,829.000000
mean,35.647042,137.415311
std,2.084659,3.650971
min,33.211967,130.195555
25%,34.695124,135.341564
50%,35.658068,139.685474
75%,35.694003,139.751599
max,44.020632,144.273398


In [53]:
air_visit_data

,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6
5,air_ba937bf13d40fb24,2016-01-19,9
6,air_ba937bf13d40fb24,2016-01-20,31
7,air_ba937bf13d40fb24,2016-01-21,21
8,air_ba937bf13d40fb24,2016-01-22,18
9,air_ba937bf13d40fb24,2016-01-23,26


欠損値なし

In [63]:
air_visit_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252108 entries, 0 to 252107
Data columns (total 3 columns):
air_store_id    252108 non-null object
visit_date      252108 non-null object
visitors        252108 non-null int64
dtypes: int64(1), object(2)
memory usage: 5.8+ MB


In [66]:
air_visit_data.describe()

,visitors
count,252108.000000
mean,20.973761
std,16.757007
min,1.000000
25%,9.000000
50%,17.000000
75%,29.000000
max,877.000000


In [54]:
date_info

,calendar_date,day_of_week,holiday_flg
0,2016-01-01,Friday,1
1,2016-01-02,Saturday,1
2,2016-01-03,Sunday,1
3,2016-01-04,Monday,0
4,2016-01-05,Tuesday,0
5,2016-01-06,Wednesday,0
6,2016-01-07,Thursday,0
7,2016-01-08,Friday,0
8,2016-01-09,Saturday,0
9,2016-01-10,Sunday,0


欠損値なし

In [67]:
date_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 3 columns):
calendar_date    517 non-null object
day_of_week      517 non-null object
holiday_flg      517 non-null int64
dtypes: int64(1), object(2)
memory usage: 12.2+ KB


holiday_flgは０、１以外の値の混入なし

In [70]:
date_info.holiday_flg.nunique()

2

In [55]:
hpg_reserve

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,hpg_c63f6f42e088e50f,2016-01-01 11:00:00,2016-01-01 09:00:00,1
1,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3
2,hpg_c8e24dcf51ca1eb5,2016-01-01 16:00:00,2016-01-01 14:00:00,2
3,hpg_24bb207e5fd49d4a,2016-01-01 17:00:00,2016-01-01 11:00:00,5
4,hpg_25291c542ebb3bc2,2016-01-01 17:00:00,2016-01-01 03:00:00,13
5,hpg_28bdf7a336ec6a7b,2016-01-01 17:00:00,2016-01-01 15:00:00,2
6,hpg_2a01a042bca04ad9,2016-01-01 17:00:00,2016-01-01 17:00:00,2
7,hpg_2a84dd9f4c140b82,2016-01-01 17:00:00,2016-01-01 15:00:00,2
8,hpg_2ad179871696901f,2016-01-01 17:00:00,2016-01-01 13:00:00,2
9,hpg_2c1d989eedb0ff83,2016-01-01 17:00:00,2016-01-01 15:00:00,6


In [71]:
hpg_reserve.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000320 entries, 0 to 2000319
Data columns (total 4 columns):
hpg_store_id        object
visit_datetime      object
reserve_datetime    object
reserve_visitors    int64
dtypes: int64(1), object(3)
memory usage: 61.0+ MB


欠損値なし

In [85]:
hpg_reserve.isnull().sum()

hpg_store_id        0
visit_datetime      0
reserve_datetime    0
reserve_visitors    0
dtype: int64

In [56]:
hpg_store_info

,hpg_store_id,hpg_genre_name,hpg_area_name,latitude,longitude
0,hpg_6622b62385aec8bf,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
1,hpg_e9e068dd49c5fa00,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
2,hpg_2976f7acb4b3a3bc,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
3,hpg_e51a522e098f024c,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
4,hpg_e3d0e1519894f275,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
5,hpg_530cd91db13b938e,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
6,hpg_02457b318e186fa4,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
7,hpg_0cb3c2c490020a29,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
8,hpg_3efe9b08c887fe9a,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
9,hpg_765e8d3ba261dc1c,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221


欠損値なし

In [86]:
hpg_store_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4690 entries, 0 to 4689
Data columns (total 5 columns):
hpg_store_id      4690 non-null object
hpg_genre_name    4690 non-null object
hpg_area_name     4690 non-null object
latitude          4690 non-null float64
longitude         4690 non-null float64
dtypes: float64(2), object(3)
memory usage: 183.3+ KB


In [87]:
hpg_store_info.describe()

,latitude,longitude
count,4690.000000,4690.000000
mean,35.810261,137.675816
std,2.138755,3.197233
min,33.311645,130.339313
25%,34.692109,135.498859
50%,35.659214,139.495733
75%,35.703381,139.737998
max,43.774846,143.714585


In [57]:
sample_submission

,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,0
1,air_00a91d42b08b08d9_2017-04-24,0
2,air_00a91d42b08b08d9_2017-04-25,0
3,air_00a91d42b08b08d9_2017-04-26,0
4,air_00a91d42b08b08d9_2017-04-27,0
5,air_00a91d42b08b08d9_2017-04-28,0
6,air_00a91d42b08b08d9_2017-04-29,0
7,air_00a91d42b08b08d9_2017-04-30,0
8,air_00a91d42b08b08d9_2017-05-01,0
9,air_00a91d42b08b08d9_2017-05-02,0


欠損値なし

In [88]:
sample_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32019 entries, 0 to 32018
Data columns (total 2 columns):
id          32019 non-null object
visitors    32019 non-null int64
dtypes: int64(1), object(1)
memory usage: 500.4+ KB


In [58]:
store_id_relation

,air_store_id,hpg_store_id
0,air_63b13c56b7201bd9,hpg_4bc649e72e2a239a
1,air_a24bf50c3e90d583,hpg_c34b496d0305a809
2,air_c7f78b4f3cba33ff,hpg_cd8ae0d9bbd58ff9
3,air_947eb2cae4f3e8f2,hpg_de24ea49dc25d6b8
4,air_965b2e0cf4119003,hpg_653238a84804d8e7
5,air_a38f25e3399d1b25,hpg_50378da9ffb9b6cd
6,air_3c938075889fc059,hpg_349b1b92f98b175e
7,air_68301bcb11e2f389,hpg_2c09f3abb2220659
8,air_5f6fa1b897fe80d5,hpg_40aff6385800ebb1
9,air_00a91d42b08b08d9,hpg_fbe603376b5980fc


欠損値なし

In [90]:
store_id_relation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 2 columns):
air_store_id    150 non-null object
hpg_store_id    150 non-null object
dtypes: object(2)
memory usage: 2.4+ KB


#### store_id_relationデータの確認

確認したところair_store_idとhpg_store_idの関係性が分かっていない...

片方情報がかけている、もしくは緯度、経緯が異なるので別の店舗情報なのでは？

In [106]:
store_id_relation.head()

,air_store_id,hpg_store_id
0,air_63b13c56b7201bd9,hpg_4bc649e72e2a239a
1,air_a24bf50c3e90d583,hpg_c34b496d0305a809
2,air_c7f78b4f3cba33ff,hpg_cd8ae0d9bbd58ff9
3,air_947eb2cae4f3e8f2,hpg_de24ea49dc25d6b8
4,air_965b2e0cf4119003,hpg_653238a84804d8e7


air_store_infoには対応する情報が存在している

In [107]:
air_store_info[air_store_info['air_store_id'] == 'air_63b13c56b7201bd9']

,air_store_id,air_genre_name,air_area_name,latitude,longitude
404,air_63b13c56b7201bd9,Cafe/Sweets,Fukuoka-ken Fukuoka-shi Daimyō,33.586994,130.394894


hpg_store_infoには対応する情報が存在していない

In [9]:
hpg_store_info[hpg_store_info['hpg_store_id'] == 'hpg_4bc649e72e2a239a']

,hpg_store_id,hpg_genre_name,hpg_area_name,latitude,longitude


同じ行に並ぶair_store_idとhpg_store_idでは店がある街の名前が一致していない

In [10]:
air_store_info[air_store_info['air_store_id'] == 'air_a24bf50c3e90d583']

,air_store_id,air_genre_name,air_area_name,latitude,longitude
172,air_a24bf50c3e90d583,Dining bar,Shizuoka-ken Hamamatsu-shi Motoshirochō,34.710895,137.72594


In [11]:
hpg_store_info[hpg_store_info['hpg_store_id'] == 'hpg_c34b496d0305a809']

,hpg_store_id,hpg_genre_name,hpg_area_name,latitude,longitude
1845,hpg_c34b496d0305a809,Italian,Shizuoka-ken Hamamatsu-shi Sukenobuchō,34.721644,137.738944


store_id_relationは150行あるが、その150行全てのair_store_idに一致する店がair_store_infoにある

In [12]:
store_id_relation.shape

(150, 2)

In [13]:
air_store_info[air_store_info.air_store_id.isin(store_id_relation["air_store_id"]) == True].shape

(150, 5)

store_id_relation の150行の内、63行のみhpg_store_idに一致する店がhpg_store_infoにある

In [14]:
hgp_existing_store_in_relation = hpg_store_info[hpg_store_info.hpg_store_id.isin\
                                                (store_id_relation["hpg_store_id"]) == True]
hgp_existing_store_in_relation.shape

(63, 5)

store_id_relationにあるあ全ての店舗情報が存在するわけではない。

air_store_infoとair_visit_dataには８２９店舗の情報があるがair_reserveには３１４店舗の情報しかない

In [16]:
air_store_info['air_store_id'].nunique()

829

In [17]:
air_visit_data['air_store_id'].nunique()

829

In [18]:
air_reserve['air_store_id'].nunique()

314

In [24]:
train_df = air_visit_data.copy()

### 前処理およびモデルの作成（ ランダムフォレストを使用）  
前処理ではair_store_idをはじめとするいくつかの情報がobject型であり、そのままでは学習に利用できないため  
カテゴリ変数化を行った。  
visit_datetimeは年・月・日がハイフンで繋がっているため情報を利用するためにそれらを取り除いた。  
特徴量エンジニアリングとして、その日の翌日が祝日である場合、通常とは異なる来客人数になる可能性を考えて、  
来客数予測の手掛かりとして、その日が何曜日であるかという条件と組み合わせてダミー変数を作成した。

#### 学習データ作成

曜日を0〜6数値に替えておく

In [77]:
#date_info.head()
date_info['day_of_week'] = date_info['day_of_week'].map({ x : i for i, x in enumerate\
                                                         (np.unique(date_info['day_of_week']))})

calendar_date列をvisit_date列に替えておく

In [78]:
date_info['visit_date'] =date_info['calendar_date']
date_info = date_info.drop(['calendar_date'], axis=1)

In [79]:
# 目的変数列（visitors）を削除しておく
X_train = air_visit_data.drop(["visitors"],axis=1)

# date_infoをマージして曜日と祝日情報を追加
X_train = pd.merge(X_train, date_info, on = 'visit_date', how='left') 

#visit_dateの値から年月日の間のハイフンを取り除く
X_train['visit_date'] = X_train['visit_date'].map(lambda x : int(x.split('-')[0] + x.split('-')[1] +\
                                                                 x.split('-')[2]))
#各air_store_idに番号をふる
X_train['air_store_id'] = X_train['air_store_id'].map({ x : i for i, x in enumerate(np.unique\
                                                                                    (X_train['air_store_id']))})
#目的変数列（visitors）をy_trainとしておく
y_train = air_visit_data["visitors"]

In [80]:
X_train

,air_store_id,visit_date,day_of_week,holiday_flg
0,603,20160113,6,0
1,603,20160114,4,0
2,603,20160115,0,0
3,603,20160116,2,0
4,603,20160118,1,0
5,603,20160119,5,0
6,603,20160120,6,0
7,603,20160121,4,0
8,603,20160122,0,0
9,603,20160123,2,0


学習実施

In [81]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score

clf = RandomForestRegressor(max_depth= 4, random_state=0)
clf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=4,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=0,
           verbose=0, warm_start=False)

#### テストデータ作成

sample_submissionのid列からair_store_idとvisit_dateを抽出する（実際に予測すべき期間のvisit_dateを抽出）

In [82]:
air_id = sample_submission['id'].map(lambda x : x[:20])
air_id.name = "air_store_id"

In [83]:
air_date = sample_submission['id'].map(lambda x : x[21:])
air_date.name = "visit_date"

In [84]:
X_test = pd.concat([air_id,air_date],axis=1)
X_test = pd.merge(X_test, date_info, on = 'visit_date', how='left') 
X_test['visit_date'] = X_test['visit_date'].map(lambda x : int(x.split('-')[0] + x.split('-')[1] +\
                                                               x.split('-')[2]))
X_test['air_store_id'] = X_test['air_store_id'].map({ x : i for i, x in enumerate(np.unique\
                                                                                  (X_test['air_store_id']))})

予測実施

In [85]:
sub_format = sample_submission.copy()

In [86]:
sub_format['visitors'] = clf.predict(X_test)

submissionを行う

In [42]:
submission_df.to_csv('submission_001.csv', index=False)

air_reserve = pd.read_csv("../input/air_reserve.csv.zip")
air_store_info = pd.read_csv("../input/air_store_info.csv.zip")
air_visit_data = pd.read_csv("../input/air_visit_data.csv.zip")
date_info = pd.read_csv("../input/date_info.csv.zip")
hpg_reserve = pd.read_csv("../input/hpg_reserve.csv.zip")
hpg_store_info = pd.read_csv("../input/hpg_store_info.csv.zip")
sample_submission = pd.read_csv("../input/sample_submission.csv.zip")
store_id_relation = pd.read_csv("../input/store_id_relation.csv.zip")

### 特徴量エンジニアリング

X_trainにvisitorsの列を追加しておく

In [87]:
X_train["visitors"] = air_visit_data["visitors"]

店舗ごとの来客数のmax,min,meanを求める

In [88]:
new_feature=X_train.groupby(["air_store_id"],as_index=False)["visitors"]\
                            .agg({'store_visit_max':'max',
                                  'store_visit_min':'min',
                                  'store_visit_mean':'mean'})

作成した特徴量をX_trainに追加する

In [89]:
X_train = pd.merge(X_train,new_feature,how='left',on='air_store_id')

店舗ごとに各曜日の来客数のmax,min,meanを求める

In [90]:
new_feature2=X_train.groupby(["air_store_id","day_of_week"],as_index=False)["visitors"]\
                            .agg({'store_dow_visit_max':'max',
                                  'store_dow_visit_min':'min',
                                  'store_dow_visit_mean':'mean'})

作成した特徴量をX_trainに追加する

In [91]:
X_train = pd.merge(X_train,new_feature2,how='left',on=['air_store_id','day_of_week'])

店舗・曜日・祝日の異なる組み合わせで来客数のmax,min,meanを求める

In [92]:
new_feature3=X_train.groupby(["air_store_id","day_of_week","holiday_flg"],as_index=False)["visitors"]\
                            .agg({'store_dow_holi_visit_max':'max',
                                  'store_dow_holi_visit_min':'min',
                                  'store_dow_holi_visit_mean':'mean'})

作成した特徴量をX_trainに追加する

In [93]:
X_train = pd.merge(X_train,new_feature3,how='left',on=['air_store_id','day_of_week','holiday_flg'])

欠損値を０で埋めておき、予測に使わないvisit_date,visitors列を削除する  

In [94]:
X_train2 = X_train.fillna(0)
X_train2 = X_train2.drop(["visit_date","visitors"],axis=1)

学習実施

In [95]:
clf2 = RandomForestRegressor(max_depth= 4, random_state=0)
clf2.fit(X_train2, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=4,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=0,
           verbose=0, warm_start=False)

店のid、曜日、祝日の組み合わせが同じ行は値が重複しているので行を減らす

In [104]:
X_train3 = X_train2.drop_duplicates(["air_store_id","day_of_week","holiday_flg"])

テストデータを作成する

In [106]:
X_test2 = X_test.drop(["visit_date"],axis=1)

In [107]:
X_test3 = pd.merge(X_test2, X_train3 ,how = 'left', on = ['air_store_id','day_of_week','holiday_flg'])

In [108]:
X_test3 = X_test3.fillna(0)

In [73]:
sub_format2 = sample_submission.copy()

予測実施

In [110]:
sub_format2['visitors'] = clf2.predict(X_test3)

In [58]:
sub_format2.to_csv('sub_features.csv', index=False)

### 時系列検証

時系列データの場合は通常のクロスバリデーションによる検証では
- ある期間だけにあてはまる学習（特定期間への過学習）
が発生してしまう。

In [111]:
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb

In [133]:
# データの読み込み
# date_infoのみcalendar_dateをvisit_dateに変更　=>結合のため
# air_visit_data が訓練データ
# sample_submissionがテストデータ　ここまではあっていた。
data = {
    'tra':
    pd.read_csv('../input/air_visit_data.csv.zip'),
    'as':
    pd.read_csv('../input/air_store_info.csv.zip'),
    'hs':
    pd.read_csv('../input/hpg_store_info.csv.zip'),
    'ar':
    pd.read_csv('../input/air_reserve.csv.zip'),
    'hr':
    pd.read_csv('../input/hpg_reserve.csv.zip'),
    'id':
    pd.read_csv('../input/store_id_relation.csv.zip'),
    'tes':
    pd.read_csv('../input/sample_submission.csv.zip'),
    'hol':
    pd.read_csv('../input/date_info.csv.zip').rename(columns={
        'calendar_date': 'visit_date'
    })
}

# ホットペッパーグルメの予約情報にリレーション情報を内部結合
data['hr'] = pd.merge(data['hr'], data['id'], how='inner', on=['hpg_store_id'])

In [134]:
# airとhpgの予約情報
for df in ['ar', 'hr']:
    # datetime型に変換
    data[df]['visit_datetime'] = pd.to_datetime(data[df]['visit_datetime'])
    data[df]['visit_datetime'] = data[df]['visit_datetime'].dt.date
    data[df]['reserve_datetime'] = pd.to_datetime(data[df]['reserve_datetime'])
    data[df]['reserve_datetime'] = data[df]['reserve_datetime'].dt.date
    # 予約日と訪問日の差をreserve_datetime_diffに格納
    data[df]['reserve_datetime_diff'] = data[df].apply(
        lambda r: (r['visit_datetime'] - r['reserve_datetime']).days, axis=1)
    # 並び替えて何をしているか分からない@@@@
    data[df] = data[df].groupby(
        ['air_store_id', 'visit_datetime'], as_index=False)[[
            'reserve_datetime_diff', 'reserve_visitors'
        ]].sum().rename(columns={
            'visit_datetime': 'visit_date'
        })
    print(data[df].head())

           air_store_id  visit_date  reserve_datetime_diff  reserve_visitors
0  air_00a91d42b08b08d9  2016-10-31                      0                 2
1  air_00a91d42b08b08d9  2016-12-05                      4                 9
2  air_00a91d42b08b08d9  2016-12-14                      6                18
3  air_00a91d42b08b08d9  2016-12-17                      6                 2
4  air_00a91d42b08b08d9  2016-12-20                      2                 4
           air_store_id  visit_date  reserve_datetime_diff  reserve_visitors
0  air_00a91d42b08b08d9  2016-01-14                      3                 2
1  air_00a91d42b08b08d9  2016-01-15                      6                 4
2  air_00a91d42b08b08d9  2016-01-16                      3                 2
3  air_00a91d42b08b08d9  2016-01-22                      3                 2
4  air_00a91d42b08b08d9  2016-01-29                      6                 5


In [135]:
# 訓練データに特徴量として、年月日、曜日、年、月を追加
data['tra']['visit_date'] = pd.to_datetime(data['tra']['visit_date'])
data['tra']['dow'] = data['tra']['visit_date'].dt.dayofweek
data['tra']['year'] = data['tra']['visit_date'].dt.year
data['tra']['month'] = data['tra']['visit_date'].dt.month
data['tra']['visit_date'] = data['tra']['visit_date'].dt.date
data['tra'].head(1)

,air_store_id,visit_date,visitors,dow,year,month
0,air_ba937bf13d40fb24,2016-01-13,25,2,2016,1


In [136]:
# テストデータのidをair_store_idとvisit_dateに分割
data['tes']['visit_date'] = data['tes']['id'].map(
    lambda x: str(x).split('_')[2])
data['tes']['air_store_id'] = data['tes']['id'].map(
    lambda x: '_'.join(x.split('_')[:2]))

# テストデータに特徴量として、年月日、曜日、年、月を追加
data['tes']['visit_date'] = pd.to_datetime(data['tes']['visit_date'])
data['tes']['dow'] = data['tes']['visit_date'].dt.dayofweek
data['tes']['year'] = data['tes']['visit_date'].dt.year
data['tes']['month'] = data['tes']['visit_date'].dt.month
data['tes']['visit_date'] = data['tes']['visit_date'].dt.date

data['tes'].head(1)

,id,visitors,visit_date,air_store_id,dow,year,month
0,air_00a91d42b08b08d9_2017-04-23,0,2017-04-23,air_00a91d42b08b08d9,6,2017,4


In [137]:
# 予測しなければならない店のair_idを保持
unique_stores = data['tes']['air_store_id'].unique()

# 店ごとに７日分の行を確保
stores = pd.concat(
    [
        pd.DataFrame({
            'air_store_id': unique_stores,
            'dow': [i] * len(unique_stores)
        }) for i in range(7)
    ],
    axis=0,
    ignore_index=True).reset_index(drop=True)

# その曜日の来客者数の 最少値　, 平均, 中央値, 最大値  を sotres　に追加
tmp = data['tra'].groupby(
    ['air_store_id', 'dow'],
    as_index=False)['visitors'].min().rename(columns={
        'visitors': 'min_visitors'
    })
stores = pd.merge(stores, tmp, how='left', on=['air_store_id', 'dow'])

tmp = data['tra'].groupby(
    ['air_store_id', 'dow'],
    as_index=False)['visitors'].mean().rename(columns={
        'visitors': 'mean_visitors'
    })
stores = pd.merge(stores, tmp, how='left', on=['air_store_id', 'dow'])
tmp = data['tra'].groupby(
    ['air_store_id', 'dow'],
    as_index=False)['visitors'].median().rename(columns={
        'visitors': 'median_visitors'
    })
stores = pd.merge(stores, tmp, how='left', on=['air_store_id', 'dow'])
tmp = data['tra'].groupby(
    ['air_store_id', 'dow'],
    as_index=False)['visitors'].max().rename(columns={
        'visitors': 'max_visitors'
    })
stores = pd.merge(stores, tmp, how='left', on=['air_store_id', 'dow'])
tmp = data['tra'].groupby(
    ['air_store_id', 'dow'],
    as_index=False)['visitors'].count().rename(columns={
        'visitors': 'count_observations'
    })
stores = pd.merge(stores, tmp, how='left', on=['air_store_id', 'dow'])

# 店のジャンルや位置情報を離散値に変換して追加
stores = pd.merge(stores, data['as'], how='left', on=['air_store_id'])
lbl = preprocessing.LabelEncoder()
stores['air_genre_name'] = lbl.fit_transform(stores['air_genre_name'])
stores['air_area_name'] = lbl.fit_transform(stores['air_area_name'])

# 祝日や曜日情報（離散値に変換）を訓練データ、テストデータに結合する
data['hol']['visit_date'] = pd.to_datetime(data['hol']['visit_date'])
data['hol']['day_of_week'] = lbl.fit_transform(data['hol']['day_of_week'])
data['hol']['visit_date'] = data['hol']['visit_date'].dt.date

train = pd.merge(data['tra'], data['hol'], how='left', on=['visit_date'])
test = pd.merge(data['tes'], data['hol'], how='left', on=['visit_date'])

# 今まで作成したstoresを訓練、テストデータに結合
# その曜日の来客者数の最少値、平均、中央値、最大値
# 店のジャンルや位置情報
train = pd.merge(data['tra'], stores, how='left', on=['air_store_id', 'dow'])
test = pd.merge(data['tes'], stores, how='left', on=['air_store_id', 'dow'])

# webからの予約情報を結合、何してるか？@@@@
for df in ['ar', 'hr']:
    train = pd.merge(
        train, data[df], how='left', on=['air_store_id', 'visit_date'])
    test = pd.merge(
        test, data[df], how='left', on=['air_store_id', 'visit_date'])

# id, air_store_id, visit_date, visitors以外の項目をcolに格納
col = [
    c for c in train
    if c not in ['id', 'air_store_id', 'visit_date', 'visitors']
]
# 欠損値を-1埋めする
train = train.fillna(-1)
test = test.fillna(-1)

# データ型の変換によるデータ圧縮
print('Binding to float32')

for c, dtype in zip(train.columns, train.dtypes):
    if dtype == np.float64:
        train[c] = train[c].astype(np.float32)

for c, dtype in zip(test.columns, test.dtypes):
    if dtype == np.float64:
        test[c] = test[c].astype(np.float32)

# 説明変数を作成、　特徴量として店のIDと訪問日、来客者数（目的変数）を省く
train_x = train.drop(['air_store_id', 'visit_date', 'visitors'], axis=1)
# 目的変数としてlog(1 + y)を取る
# 先にlog(1+y)計算しておくと評価式でrmseを使えばrmsleになる
train_y = np.log1p(train['visitors'].values)
print(train_x.shape, train_y.shape)
# テストデータの説明変数を作成
test_x = test.drop(['id', 'air_store_id', 'visit_date', 'visitors'], axis=1)

# parameter tuning of lightgbm
# start from default setting
# 他のカーネルのパラメーターを見たい
gbm0 = lgb.LGBMRegressor(
    objective='regression', # 回帰
    num_leaves=60, 
    learning_rate=0.01,
    n_estimators=10000)

gbm0.fit(train_x, train_y, eval_metric='rmse')
predict_y = gbm0.predict(test_x)
# 予測値がlog(x+1)になっているのでexp(y) - 1で元の値に戻す
test['visitors'] = np.expm1(predict_y)
test[['id', 'visitors']].to_csv(
    'gbm0_submission.csv', index=False, float_format='%.3f')  # LB0.493

Binding to float32
(252108, 16) (252108,)


### 時系列検証の流れ
date_info.csvからテストデータの期間（2017/04/23~）を省く  
分割された日付インデックスから訓練データ、テストデータを作成（visit_dateで範囲抽出）

In [174]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error


# 日付データからテストデータの日付を引いたものを用意する
#TimeSeriesSplit に対してはindexがあるものを渡せば良い
# 渡したデータの列数は関係ない
valid_day = data['hol'].iloc[:-39,:]
#n_splits=2 とすれば２セットに分割する
tscv = TimeSeriesSplit(n_splits=2)

for train_index, test_index in tscv.split(valid_day):
    #print("TRAIN:", type(train_index), "TEST:", test_index)
    # 学習する範囲の初日と最終日を出力
    # train_index , test_index は arrayでありvalid_dayの複数行を取得することになる
    #よって、0行目の０番目の要素を取得すれば初日になる
    train_start_day = np.array(valid_day)[train_index][0, 0]
    train_end_day = np.array(valid_day)[train_index][-1, 0]
    
    # テストする範囲の初日と最終日を出力
    test_start_day = np.array(valid_day)[test_index][0, 0]
    test_end_day = np.array(valid_day)[test_index][-1, 0]

    ts_train = train[(train_start_day <= train['visit_date']) & (train['visit_date'] <= train_end_day) ]
    ts_test = train[(test_start_day <= train['visit_date']) & (train['visit_date'] <= test_end_day) ]
    # TODO　上記２行を下記のような条件式で行う。
    # test_df =train[(train['visit_date'] - datetime.date(2017, 1, 29) >= datetime.timedelta(42)) & \
    #(train['visit_date'] - datetime.date(2017, 1, 29) <= datetime.timedelta(80))]


    # 学習データとテストデータを作成
    
    # 説明変数を作成、　特徴量として店のIDと訪問日、来客者数（目的変数）を省く
    train_x = ts_train.drop(['air_store_id', 'visit_date', 'visitors'], axis=1)
    # 目的変数としてlog(1 + y)を取る
    # 先にlog(1+y)計算しておくと評価式でrmseを使えばrmsleになる
    train_y = np.log1p(ts_train['visitors'].values)
    # テストデータに対して同じ処理を施す
    test_x = ts_test.drop(['air_store_id', 'visit_date', 'visitors'], axis=1)
    test_y = np.log1p(ts_test['visitors'].values)

    # lightgbmのパラメータ設定
    gbm0 = lgb.LGBMRegressor(
        objective='regression', # 回帰
        num_leaves=60, 
        learning_rate=0.01,
        n_estimators=10000)
    
    #学習実施
    gbm0.fit(train_x, train_y, eval_metric='rmse')
    #予測実施
    predict_y = gbm0.predict(test_x)
    #評価計算
    print(np.sqrt(mean_squared_error(test_y, predict_y) ))

0.5686939327390773
0.551332108179359


### Bayesian Hyperparameter Optimization

In [120]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from bayes_opt import BayesianOptimization
from tqdm import tqdm

# 最小化する関数を定義する
def lgb_evaluate(min_child_weight,
                 colsample_bytree,
                 max_depth,
                 subsample):

    lgb_reg = lgb.LGBMRegressor(max_depth=int(max_depth),
                                min_child_weight = min_child_weight,
                                colsample_bytree = colsample_bytree,
                                subsample=subsample
                               )

    lgb_reg.fit(train_x, train_y, eval_metric='rmse')

    predict_y = lgb_reg.predict(test_x)
    print(np.sqrt(mean_squared_error(test_y, predict_y) ))
    # 評価関数の値を０にしたいが、予測誤差によりプラスの値になってしまう。そこで、返り値にマイナスをつけておく
    # すると、ベイジアン最適化では、その値を最大化、つまり、マイナスの値をできる限りプラス方向へと動かそうとするので、
    # 結局、予測誤差の最小化をすることになる
    return -np.sqrt(mean_squared_error(test_y, predict_y) )

In [121]:
train_df = train # [train['visit_date'] <= pd.to_datetime('2017-02-01').date ]
test_df = train # [pd.to_datetime('2017-02-01').date < train['visit_date']]

# 説明変数を作成、　特徴量として店のIDと訪問日、来客者数（目的変数）を省く
train_x = train_df.drop(['air_store_id', 'visit_date', 'visitors'], axis=1)
# 目的変数としてlog(1 + y)を取る
# 先にlog(1+y)計算しておくと評価式でrmseを使えばrmsleになる
train_y = np.log1p(train_df['visitors'].values)

#テストデータに対しても同じ処理を施す
test_x = test_df.drop(['air_store_id', 'visit_date', 'visitors'], axis=1)
test_y = np.log1p(test_df['visitors'].values)

lgtrain = lgb.Dataset(train_x, label=train_y)
lgvalid = lgb.Dataset(test_x, label=test_y)


random_state = 2016
num_iter = 25
init_points = 5
params = {
    'eval_metric': 'mse',
    'seed': random_state
}

# パラメータの範囲を指定してインスタンスを生成する
lgbBO = BayesianOptimization(lgb_evaluate, {'min_child_weight': (1, 20),
                                            'colsample_bytree': (0.1, 1),
                                            'max_depth': (5, 15),
                                            'subsample': (0.5, 1)
                                            })

#最適化実施
lgbBO.maximize(init_points=init_points, n_iter=num_iter)

Initialization
---------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |   max_depth |   min_child_weight |   subsample | 
0.5028437275632482
    1 | 00m01s |   -0.50284 |             0.9228 |      5.7108 |             4.7758 |      0.7058 | 
0.5008485120237647
    2 | 00m01s |   -0.50085 |             0.5961 |      7.2762 |             3.6876 |      0.6176 | 
0.49954196190371614
    3 | 00m01s |   -0.49954 |             0.6285 |     11.3015 |             7.4704 |      0.7973 | 
0.5019420299169922
    4 | 00m02s |   -0.50194 |             0.6869 |      6.5270 |            11.0401 |      0.9148 | 
0.500530744832615
    5 | 00m01s |   -0.50053 |             0.5926 |      8.8760 |            12.8757 |      0.7789 | 


/Users/h_t_mac_book_pro/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-1.1443156e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 53, 'nit': 3, 'warnflag': 2}
  " state: %s" % convergence_dict)
/Users/h_t_mac_book_pro/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:308: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


Bayesian Optimization
---------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |   max_depth |   min_child_weight |   subsample | 
0.5266649436633826
    6 | 00m08s |   -0.52666 |             0.1122 |     14.7276 |            19.8451 |      0.8684 | 
0.4990978504822983
    7 | 00m10s |   -0.49910 |             1.0000 |     15.0000 |             1.0000 |      0.5000 | 
0.526711337644123
    8 | 00m11s |   -0.52671 |             0.1000 |      5.3667 |            11.0940 |      0.5000 | 
0.5028249784845438
    9 | 00m07s |   -0.50282 |             0.9851 |      5.4019 |            19.9863 |      0.9434 | 
0.49951133368496936
   10 | 00m08s |   -0.49951 |             0.9739 |      8.1721 |             1.0665 |      0.9817 | 
0.49953065817718606
   11 | 00m07s |   -0.49953 |             0.9754 |      9.7360 |            19.9695 |      0.5169 | 
0.49929103604990255
   12 | 00m07s |   -0.49929 |       

ベストスコア

In [175]:
lgbBO.res['max']

{'max_params': {'colsample_bytree': 0.986033255659363,
  'max_depth': 14.720046532524684,
  'min_child_weight': 19.98494545937381,
  'subsample': 0.949182292267641},
 'max_val': -0.49835867353740504}